In [4]:
#20220426 水平画角を取得する
import numpy as np
import math
#カメラ-模造紙間の距離 [mm]
d = 1000
#模造紙の実際のサイズ [mm]
Imi = [1091, 788]
#pixel size [pixel]
px = [4000, 3000]
#画像中の模造紙のサイズ [pixel]
Rpx = [3326, 2397]
#中心座標(正規化)
center = [0.5, 0.5]

#画像を丁度埋め尽くすようなサイズ[mm]を求める
Fillx = Imi[0] * px[0] / Rpx[0]
Filly = Imi[1] * px[1] / Rpx[1]
print('x:', Fillx ,'y:', Filly)
#水平画角を求める[deg]
Half_angle = math.degrees(math.atan(Fillx / 2 / d))
Horizontal_angle = 2 * Half_angle
print('Half_angle:', Half_angle)
print('Horizontal_ange:', Horizontal_angle)

x: 1312.086590499098 y: 986.2327909887359
Half_angle: 33.26660900930381
Horizontal_ange: 66.53321801860761


In [8]:
#(原点の配置されたカメラについて)角度から直線の方程式を求める
#カメラの位置座標
cam1 = np.array([0, 0, 0]) #(x,y,theta)
#cam2 = [1.1, 1.1]

def calc_equation(coordinate_bottle, cam: np.ndarray):
    #YOLOの出力結果(x座標)から中心と各ボトルのなす角を計算する
    angle = math.degrees(math.atan(abs(coordinate_bottle - center[0]) / (center[0] / (math.tan(math.radians(Half_angle))))))
    
    #カメラ位置が原点である場合(cam1[0,0])
    if cam[0] == 0 and cam[1] == 0:  
        if coordinate_bottle <= 0.5:
            slope = math.tan(math.radians(90 + angle))
        else:
            slope = math.tan(math.radians(90 - angle))
    
#     カメラ位置が原点でない場合
#     90+-angleから原点に対して回転している角度を引く？
#         #画像2について (Cam1とCam2は90度)
#         # m_A2x = math.tan(math.radians(angle_A2x))
    
#     else:
#         rot = math.degrees(np.arctan2(cam[0], cam[1])) #間違ってる　　なす角45度が出てきてしまう
            #カメラの位置ではなく向き
#         　slope = math.tan(math.radians(90 + angle - rot) ?? 場合分けも必要
    
    else:
        if coordinate_bottle <= 0.5:
            slope = math.tan(math.radians(cam[2] - angle)) #修正
        else:
            slope = math.tan(math.radians(cam[2] + angle))
    
    #切片を求める
    intercept = cam[1] - slope * cam[0]
    return(slope, intercept)

カメラの角度+90 +- angle
左側(<0.5)のとき+
右側(>0.5)のとき-

In [ ]:
calc_equation(0.954349,[,])

In [10]:
calc_equation(0.191375,[1.1,1.1,0])

(-0.40494272399278436, 1.545436996392063)

In [37]:
calc_equation(0.191375, np.array([1.1,1.1]))

(0.40494272399278436, 0.6545630036079373)

In [36]:
calc_equation(0.490500, [0,0])

(-80.22577065946315, 0.0)

In [41]:
#2直線の交点を求める
def calc_intersection(s1, i1, s2, i2):
    X = (i2 - i1)/(s1 - s2)
    Y = (s1 * i2 - s2 * i1) / (s1 - s2)

#ボトルの大きさ(高さ)を推定する
def calc_size(h1,h2):
    size = Filly * (Y - cam1[1]) * h1
    return(size)
    
    '''if:#
        size = Filly * (Y - cam1[1]) * h1
    else:
        size = Filly + (Y - cam1[1]) * h2'''
    

In [38]:
calc_intersection(-80.22577065946315,0.0,0.40494272399278436,0.6545630036079373)

(-0.008118035623659044, 0.6512756641490223)

In [ ]:
#ボトルの大きさ(高さ)を推定する
#Cam1画像の出力結果を用いる場合
print('Cam1について')
cam1_Ah = 0.373667 #cam1で検出されたBottle_AのHeight
cam1_Bh = 0.371333 #cam1で検出されたBottle_BのHeight

#A1とA2の組
cam1_A1A2h = Filly * (A1A2_y - cam1y) * cam1_Ah
print('A1とA2の組で推定した高さ:', cam1_A1A2h)

#----------------------------
#B1とA2の組
cam1_B1A2h = Filly * (B1A2_y - cam1y) * cam1_Bh
print('B1とA2の組で推定した高さ:', cam1_B1A2h)
#A1とB2の組
cam1_A1B2h = Filly * (A1B2_y - cam1y) * cam1_Ah
print('A1とB2の組で推定した高さ:', cam1_A1B2h)
#B1とB2の組
cam1_B1B2h = Filly * (B1B2_y - cam1y) * cam1_Bh
print('B1とB2の組で推定した高さ:', cam1_B1B2h)
#----------------------------

#Cam2画像の出力結果を用いる場合
print('Cam2について')
cam2_Ah = 0.190333
cam2_Bh = 0.277333

#A1とA2の組
cam2_A1A2h = Filly * (cam2x - A1A2_x) * cam2_Ah
print('A1とA2の組で推定した高さ:', cam2_A1A2h)

#----------------------------
#B1とA2の組
cam2_B1A2h = Filly * (cam2x - B1A2_x) * cam2_Ah
print('B1とA2の組で推定した高さ:', cam2_B1A2h)
#A1とB2の組
cam2_A1B2h = Filly * (cam2x - A1B2_x) * cam2_Bh
print('A1とB2の組で推定した高さ:', cam2_A1B2h)
#B1とB2の組
cam2_B1B2h = Filly * (cam2x - B1B2_x) * cam2_Bh
print('B1とB2の組で推定した高さ:', cam2_B1B2h)
#----------------------------


In [ ]:
#尤もらしい組み合わせを判定する
cam1_data = [cam1_A1A2h,cam1_B1A2h,cam1_A1B2h,cam1_B1B2h]
cam2_data = [cam2_A1A2h,cam2_B1A2h,cam2_A1B2h,cam2_B1B2h]

evaluation = []

for i in range(len(cam1_data)):
    ls = []
    ls += [cam1_data[i], cam2_data[i]]
    evaluation.append(np.var(ls))

print(evaluation)
index = np.argsort(evaluation)
print(index)

